In [1]:
import sys

sys.path.append('../GraphStructureLearning')

In [2]:
from glob import glob
import yaml
from easydict import EasyDict as edict

In [3]:
import pickle
import os
from os import path
import torch
import numpy as np

from torch_geometric.data import Data

In [4]:
from torch_geometric.utils import dense_to_sparse
from dataset.make_traffic_dataset import TrafficDatasetLoader
from utils.utils import build_fully_connected_edge_idx

In [5]:
from models.GTS.self_attention_graph_learning import Attention_Graph_Learning
from models.GTS.gts_graph_learning import GTS_Graph_Learning

from torch.nn import functional as F


In [6]:
config_file = glob('./config/GTS/metr_la.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

In [11]:
model_1 = Attention_Graph_Learning(config)
model_2 = Attention_Graph_Learning(config)

In [12]:
metr = TrafficDatasetLoader(raw_data_dir='./data/METR-LA', dataset_name='METR-LA')
data_list, X = metr.get_dataset(12, 3, 1)

In [13]:
out_1 = model_1(X, _)
out_2 = model_2(X, _)

In [14]:
out = torch.stack([out_1, out_2], dim=-1)

In [16]:
out.shape

torch.Size([207, 207, 2])

In [20]:
edge_probability = F.gumbel_softmax(out, hard=True, dim=-1)

In [21]:
edge_probability.shape

torch.Size([207, 207, 2])

In [22]:
edge_probability

tensor([[[1., 0.],
         [1., 0.],
         [1., 0.],
         ...,
         [0., 1.],
         [0., 1.],
         [0., 1.]],

        [[0., 1.],
         [0., 1.],
         [0., 1.],
         ...,
         [0., 1.],
         [0., 1.],
         [0., 1.]],

        [[0., 1.],
         [0., 1.],
         [0., 1.],
         ...,
         [0., 1.],
         [0., 1.],
         [0., 1.]],

        ...,

        [[0., 1.],
         [1., 0.],
         [1., 0.],
         ...,
         [0., 1.],
         [1., 0.],
         [0., 1.]],

        [[1., 0.],
         [1., 0.],
         [0., 1.],
         ...,
         [1., 0.],
         [1., 0.],
         [1., 0.]],

        [[0., 1.],
         [0., 1.],
         [1., 0.],
         ...,
         [1., 0.],
         [1., 0.],
         [1., 0.]]], grad_fn=<AddBackward0>)

In [29]:
edge_probability[:,:,0].shape

torch.Size([207, 207])

In [30]:
edge_probability[:,:,0]

tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 1., 1.,  ..., 0., 1., 0.],
        [1., 1., 0.,  ..., 1., 1., 1.],
        [0., 0., 1.,  ..., 1., 1., 1.]], grad_fn=<SelectBackward0>)

In [26]:
init_edge_index = build_fully_connected_edge_idx(207)

In [ ]:
adj_matrix

In [ ]:
from torch_geometric.utils import to_undirected, to_dense_adj, dense_to_sparse


In [31]:
a, b =dense_to_sparse(edge_probability[:,:,0])

In [32]:
a.shape

torch.Size([2, 21324])

In [33]:
a

tensor([[  0,   0,   0,  ..., 206, 206, 206],
        [  0,   1,   2,  ..., 204, 205, 206]])

In [34]:
init_edge_index.shape

torch.Size([2, 42642])